In [1]:
# import library
%run lib.ipynb

## Experiment 3.1: Compute prediction error
So far, we computed the error (MSE / MAE) of λ comparing the learned one with the actual one.  
In this experiment, we test the error in prediction. So,

### Part I
- M=256 sample points, generating histogram H of size K; H[k] = number of samples in bin k, i.e., count how many sample points. Therefore K is computed, but will be small for typical values of λ.
- λ = 0.5 ... 1.5
- N = 10K

Let V = (H[k] - M * Poisson(λ,k))

Then V is parameterized by

k: the current bin

i = 1,...,N: sample number

All computation must include therefore K*N values of V.

**Objective**: compute MAE(V), MSE(V), MEAN_BIAS(V), with respect to all these values.

### Part II
Repeat with λ computed by the expectation of the sample, i.e., $\sum_{k=0}^{K}\; \frac{k*H[k]}{M}$

### Part III
- Combine Part I and Part II, to find out which is better; ours should be. We need a single table/graph/plot.


- Show that the expectation of the sample, gives a biased estimate, and that ours does not.

  To do so: Same data as stage I, but NO LEARNING. Use N=10,000, compute λ by its expectation; 
  
  compare with the λ you selected at random, and show that there is bias between the two.

In [2]:
def generate_poisson_data(N, M, min_lambda=0.5, max_lambda=1.5):
    
    # define λ generator for 0.5 <= λ <= 1.5
    lambda_gen = lambda: next_lambda(min_lambda=min_lambda, max_lambda=max_lambda)

    raw, H, lambdas = generate_data(N=N, 
                                    M=M, 
                                    nextConfig=lambda_gen,
                                    sample=sample_poisson, 
                                    density=False, 
                                    dense_histogram=True, 
                                    apply_log_scale=False)
    return H, lambdas

def compute_V(H, M, lambdas):
    i,j = np.ix_(np.arange(H.shape[0]), np.arange(H.shape[1]))
    V = H - M * poisson.pmf(k=j, mu=lambdas[i])
    return V

def experiment_3_1_part_I(H_train, H_test, lambdas_train, lambdas_test):
    
    # fit model
    print(f'fitting dnn model ... ', end='')
    start_time = time.time()
    dnn_model, history = dnn_fit(X_train=H_train, y_train=lambdas_train)
    train_time = round(time.time() - start_time)
    print(f'duration: {round(train_time)} sec.')

    # predict lambdas
    print(f'predicting lambdas ... ', end='')
    lambdas_pred, sqrt_mse = dnn_predict(dnn_model, H_test, lambdas_test)
    print(f'sqrt_mse(λ): {sqrt_mse:.6f}')
    
    # compute V: V[i,j] = H[i,j] - M * Poisson(λ[i], j)
    V = compute_V(H=H_test, M=M, lambdas=lambdas_pred)
    
    # MAE(V)
    MAE = np.mean(np.abs(V))
    print(f'MAE: {MAE:.6f}')
    
    # SQRT_MSE(V)
    SQRT_MSE = np.sqrt(np.mean(np.square(V)))
    print(f'SQRT_MSE: {SQRT_MSE:.6f}')
    
    # MEAN_BIAS(V)
    MEAN_BIAS = np.mean(V)
    print(f'MEAN_BIAS: {MEAN_BIAS:.6f}')


In [3]:
# for reporoducable results
reset_random_state(17)

# generate data (histogram and lambdas)
N=10000
M=256
min_lambda=0.5
max_lambda=1.5
print(f'generating data (M={M}, N={N}) ... ', end='')
H, lambdas = generate_poisson_data(N, M, min_lambda=min_lambda, max_lambda=max_lambda)
H_train, H_test, lambdas_train, lambdas_test = train_test_split(H, lambdas, test_size=0.25, 
                                                                random_state=RANDOM_STATE)
print(f'histogram shape: {H_train.shape}')

print()
print(f'experiment_3_1_part_I (prediced λ)')
experiment_3_1_part_I(H_train, H_test, lambdas_train, lambdas_test)

generating data (M=256, N=10000) ... histogram shape: (7500, 11)

experiment_3_1_part_I (prediced λ)
fitting dnn model ... duration: 23 sec.
predicting lambdas ... sqrt_mse(λ): 0.060201
MAE: 1.782004
SQRT_MSE: 3.399430
MEAN_BIAS: 0.000002


### Part II
Repeat with λ computed by the expectation of the sample, i.e., $\sum_{k=0}^{K}\; \frac{k*H[k]}{M}$

In [4]:
def calc_lambdas_expected(H, M):
    """ return lambda expected at each row """
    k = np.ix_(np.arange(H_test.shape[1]))
    return np.sum(k*H/M, axis=1)

def experiment_3_1_part_II(H, M):
    
    lambdas_expected = calc_lambdas_expected(H, M)
    
    # compute V: V[i,j] = H[i,j] - M * Poisson(λ[i], j)
    V = compute_V(H=H, M=M, lambdas=lambdas_expected)
    
    # MAE(V)
    MAE = np.mean(np.abs(V))
    print(f'MAE: {MAE:.6f}')
    
    # SQRT_MSE(V)
    SQRT_MSE = np.sqrt(np.mean(np.square(V)))
    print(f'SQRT_MSE: {SQRT_MSE:.6f}')
    
    # MEAN_BIAS(V)
    MEAN_BIAS = np.mean(V)
    print(f'MEAN_BIAS: {MEAN_BIAS:.6f}')

print(f'experiment_3_1_part_II (expected λ)')
experiment_3_1_part_II(H=H_test, M=M)

experiment_3_1_part_II (expected λ)
MAE: 1.754229
SQRT_MSE: 3.370083
MEAN_BIAS: 0.000002


## Experiment 3.2: Can we help learning of Poisson, by teaching the underlying mathematics.

Learning the computed λ, by using expectation is **extremely easy** for a neural network. 
Based on #7, we hope that the DNN does better than this. So force it to do better, we give it the computed λ as input, and only ask it to learn the difference between the *real* λ (as used to synthesize the data) and the *estimator* λ. 

Our question is: _can we help the neural network to learn the data if we give it mathematical hints on it?_

To do so, define ζ to be the estimator λ, as computed by the expectation; 

> $ζ = Σ_{i=0}^{K} \frac{k * H[k]}{M}$

Define a new histogram Z, computed from H, using ζ .  The new histogram will show the difference between the expected value and the real value,  i.e, run the following loop for k=0, ... , K

> $Z[k] = H[k] - M * Poisson(ζ,k)$

Now apply DNN. The new data includes all the data the previous DNN achieved, but in a slightly more convenient way. Hopefully, the results are better.

Design and implement and experiment to give a conclusive answer to this question. 


In [5]:
# TODO